# 基于LSTM深度神经网络的股指预测模型

In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
from sklearn.metrics import mean_squared_error
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
import seaborn as sns
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 提取主力合约

In [3]:
path = '/home/chocolate/LSTM-source/Minute_data_IF/'
new_path = '/home/chocolate/LSTM-source/Main_product/'
file_list = os.listdir(path)
for i in range(len(file_list)):
    train_data = pd.read_csv(path + file_list[i],encoding="gbk")
    train_data = train_data.sort_values(by='Time')  # 对原数据重新按时间排序，因为有些数据顺序被打乱了。
#     train_data = train_data[:-5] # 最后5分钟的数据不可取，去除
    newtime = pd.to_datetime(train_data['Time']) # 将时间转换为datetime格式
    newtime_list = list(train_data['Time']) # 将时间Series格式转换为list
    main_month = newtime.iloc[-1].month # 提取主力合约的月份
    if i == 0:
        for j in range(len(train_data)):
            if newtime.iloc[j].month != main_month:
                 train_data.drop(j,inplace=True) # 如果不是主力合约的数据直接删除
        train_data.to_csv(new_path + file_list[i].split('.')[0]+ '_main' + '.csv',sep=',') # 将提取的主力合约另存为csv文件
        main_last_time = train_data['Time'].iloc[-10] # 取上一个合约最后一个交割日的时间
        print "第一个合约交割日时间：",main_last_time
    else:
        main_last_index = newtime_list.index(main_last_time) # 返回上一个合约的交割日在当前合约中的索引
        print "上个合约交割时间所在当前合约的索引：",main_last_index
        for k in range(len(train_data)):
            if k< main_last_index:
                train_data.drop(k,inplace=True) # 如果不是主力合约的数据直接删除
        print "当前主力合约起始时间：",train_data['Time'].iloc[1]
        train_data.to_csv(new_path + file_list[i].split('.')[0]+ '_main' + '.csv',sep=',') # 将提取的主力合约另存为csv文件
        main_last_time = train_data['Time'].iloc[-10] # 取上一个合约最后一个交割日的时间
        print "当前合约交割日时间：",main_last_time

第一个合约交割日时间： 2016-05-20 14:51:00.000000


ValueError: u'2016-05-20 14:51:00.000000' is not in list

In [6]:
# 提取训练集的主力合约
trainset_path = '/home/chocolate/LSTM-source/Minute_data_IF/' # 预处理后的合约
trainset_main_path = '/home/chocolate/LSTM-source/Main_product/'
file_list = os.listdir(trainset_path)
for i in range(len(file_list)):
    train_data = pd.read_csv(trainset_path + file_list[i],header=0, sep=',',encoding="gbk")
    train_data = train_data.sort_values(by='Time')  # 对原数据重新按时间排序，因为有些数据顺序被打乱了。
    train_data = train_data[:-5] # 最后5分钟的数据不可取，去除
    newtime = pd.to_datetime(train_data['Time']) # 将时间转换为datetime格式
    newtime_list = list(train_data['Time']) # 将时间Series格式转换为list
    main_month = newtime.iloc[-1].month # 提取主力合约的月份
    if i == 0:
        for j in range(len(train_data)):
            if newtime.iloc[j].month != main_month:
                 train_data.drop(j,inplace=True) # 如果不是主力合约的数据直接删除
        train_data.to_csv(trainset_main_path + file_list[i].split('.')[0]+ '_main' + '.csv',sep=',') # 将提取的主力合约另存为csv文件
        main_last_time = train_data['Time'].iloc[-10] # 取上一个合约最后一个交割日的时间
        print "第一个合约交割日时间：",main_last_time
    else:
        main_last_index = newtime_list.index(main_last_time) # 返回上一个合约的交割日在当前合约中的索引
        print "上个合约交割时间所在当前合约的索引：",main_last_index
        for k in range(len(train_data)):
            if k< main_last_index:
                train_data.drop(k,inplace=True) # 如果不是主力合约的数据直接删除
        print "当前主力合约起始时间：",train_data['Time'].iloc[1]
        train_data.to_csv(trainset_main_path + file_list[i].split('.')[0]+ '_main' + '.csv',sep=',') # 将提取的主力合约另存为csv文件
        main_last_time = train_data['Time'].iloc[-10] # 取上一个合约最后一个交割日的时间
        print "当前合约交割日时间：",main_last_time

第一个合约交割日时间： 2016-05-20 14:46:00.000000


ValueError: u'2016-05-20 14:46:00.000000' is not in list

In [59]:
path = '/home/chocolate/LSTM-source/Minute_data_IC/'
new_path = '/home/chocolate/LSTM-source/Main_product/'
file_list = os.listdir(path)
data = pd.DataFrame()
for i in range(len(file_list)):
    data_per = pd.read_csv(path + file_list[i],header=0, sep=',',encoding="gbk")
#     print data_per['Time']
    data = data.append(data_per)
#     print data_per.shape
data['Time'] = pd.to_datetime(data['Time']) 
data = data.sort_values(by='Time')  # 对原数据重新按时间排序，因为有些数据顺序被打乱了。

dataVolume = data.groupby('Time')[['Volume']].max()
data1 = pd.merge(data,dataVolume,left_on='Time',right_index=True,how='right')
data1 = data1.drop_duplicates('Time')
data1.drop('Volume_x',axis=1,inplace=True)
data1.index = range(len(data1))
data1 = DataFrame(data1,columns=['Time','Latestprice','MaxPrice','MinPrice','Stockup','Volume_y','First.Latestprice','Last.Buy1price','Last.Buy1quantity','Last.Sell1price','Last.Sell1quantity'])
data1.to_csv(new_path + 'IC_main.csv')

# 提取完之后，手动将列名”Volume_y“改为”Volume“


Int64Index([2381], dtype='int64')


Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


IndexError: indices are out-of-bounds

In [61]:
# 在处理IH合约的时候，某条数据日期格式有错误，找出并删除，重新保存
path = '/home/chocolate/LSTM-source/Minute_data_IH/'
new_path = '/home/chocolate/LSTM-source/Main_product/'
file_list = os.listdir(path)
data = pd.DataFrame()
for i in range(len(file_list)):
    data_per = pd.read_csv(path + file_list[i],header=0, sep=',',encoding="gbk")
#     print data_per['Time']
    data = data.append(data_per)
data['Time'] = pd.to_datetime(data['Time'], dayfirst=True, errors='coerce') 
idx = data[data['Time'].isnull()].index
print idx
print data.iloc[idx]
# print data.iloc[idx + 1]
data = data.drop(data.iloc[idx])

data['Time'] = pd.to_datetime(data['Time']) 
data = data.sort_values(by='Time')  # 对原数据重新按时间排序，因为有些数据顺序被打乱了。

dataVolume = data.groupby('Time')[['Volume']].max()
data1 = pd.merge(data,dataVolume,left_on='Time',right_index=True,how='right')
data1 = data1.drop_duplicates('Time')
data1.drop('Volume_x',axis=1,inplace=True)
data1.index = range(len(data1))
data1 = DataFrame(data1,columns=['Time','Latestprice','MaxPrice','MinPrice','Stockup','Volume_y','First.Latestprice','Last.Buy1price','Last.Buy1quantity','Last.Sell1price','Last.Sell1quantity'])
data1.to_csv(new_path + 'IH_main.csv')

                          Date                Time Latestprice MaxPrice  \
2381  2015-05-28T05:41:00.000Z 2015-05-28 13:41:00      3301.6   3306.4   

     MinPrice Stockup     Turnover Position Volume First.Latestprice  \
2381   3301.6       2  2.08219e+07     5807     21            3306.2   

     Last.Buy1price Last.Buy1quantity Last.Sell1price Last.Sell1quantity  
2381         3300.8                 1            3302                  6  
                          Date                Time Latestprice MaxPrice  \
2382  2015-05-28T05:42:00.000Z 2015-05-28 13:42:00      3300.2   3304.2   

     MinPrice Stockup     Turnover Position Volume First.Latestprice  \
2382     3298       5  2.67357e+07     5812     27              3302   

     Last.Buy1price Last.Buy1quantity Last.Sell1price Last.Sell1quantity  
2382         3300.6                 1          3302.4                  1  


## 数据清洗

In [64]:
# 读取已经提取好的主力合约文件

# 选择2016年1-8月的主力合约数据作为训练集，在主力合约中进行提取
# 删除非正常交易时间的数据，交易时间为上午9：30-11：30，下午1:00到3:00
# 因为每天开盘的前10分钟和收盘的最后10分钟数据波动较大，暂时在训练时去掉这些数据

# 一分钟之内，最大股价和最低股价相差超过50点的，那么应该尽量删掉这些数据。
path = '/home/chocolate/LSTM-source/Main_product/'
new_path = '/home/chocolate/LSTM-source/Main_product_cleaned/'
file_list = os.listdir(path)
for i in range(len(file_list)):
    data_temp = pd.read_csv(path + file_list[i],header=0, sep=',',encoding="gbk")
    data_temp = data_temp.sort_values(by='Time')
    data_temp['Time'] = pd.to_datetime(data_temp['Time']) 
    data_temp.index = range(len(data_temp))
    delete_list = []
    for k in range(len(data_temp)):
        if data_temp['Time'].iloc[k].hour < 9:
            delete_list.append(k)
        elif data_temp['Time'].iloc[k].hour == 9 and data_temp['Time'].iloc[k].minute <= 29:
            delete_list.append(k)
        elif data_temp['Time'].iloc[k].hour >= 15: 
            delete_list.append(k)
        elif data_temp['MaxPrice'].iloc[k] - data_temp['MinPrice'].iloc[k] > 50:
            delete_list.append(k)

    for j in range(len(data_temp)):
        if j in delete_list:
            data_temp.drop(j,inplace=True)
    data_temp.index = range(len(data_temp))
    data_temp.to_csv(new_path + file_list[i].split('.')[0]+ '_cleaned' + '.csv',sep=',')

## （1）添加用于训练的诸多特征，如移动平均线、布林带等
## （2）添加用于预测标签的诸多特征，如后1-5分钟的涨跌幅度

In [65]:
import seaborn as sns
import datetime
from pandas import Series,DataFrame
path = '/home/chocolate/LSTM-source/Main_product_cleaned/'
new_path = '/home/chocolate/LSTM-source/Main_product_added_features/'
file_list = os.listdir(path)
ma_list = [5, 12, 26]
for j in range(len(file_list)):
    newdata1 = pd.read_csv(path + file_list[j], header=0, sep=',',encoding="gbk")
    newTime = pd.to_datetime(newdata1['Time']) # 将时间转换成datetime格式
#  添加收盘价的移动平均线MA和指数平滑移动平均线EMA
    for ma in ma_list:
        newdata1['MA_' + str(ma)] = pd.rolling_mean(newdata1['Latestprice'], ma)
    # 计算移动平均线之间的距离
    newdata1['Dis_MA5_26'] = newdata1['MA_5'] - newdata1['MA_26']
    # 计算指数平滑移动平均线
    for ma in ma_list:
        newdata1['EMA_' + str(ma)] = pd.ewma(newdata1['Latestprice'], span=ma)
    # 计算指数平滑移动平均线之间的距离
    newdata1['Dis_EMA5_26'] = newdata1['EMA_5'] - newdata1['EMA_26']
    
# 添加成交量的移动平均线MA和指数平滑移动平均线EMA
    for ma in ma_list:
        newdata1['Vol_MA_'+str(ma)] =  pd.rolling_mean(newdata1['Volume'], ma)
# 计算成交量移动平均线之间的距离
    newdata1['Dis_Vol_MA5_26'] = newdata1['Vol_MA_5'] - newdata1['Vol_MA_26']
# 添加成交量的指数平滑移动平均线
    for ma in ma_list:
        newdata1['Vol_EMA_'+str(ma)] = pd.ewma(newdata1['Volume'], span=ma)
# 计算指数平滑移动平均线之间的距离
    newdata1['Dis_Vol_EMA5_26'] = newdata1['Vol_EMA_5'] - newdata1['Vol_EMA_26']  
    newdata1['DIFF_12_26'] = newdata1['EMA_12'] - newdata1['EMA_26']
# 计算离差平均值DEA，也就是计算离差值的指数平滑移动平均，设置为5分钟的指数平滑曲线
    newdata1['DEA_12_26'] =  pd.ewma(newdata1['DIFF_12_26'],span = 9)
# 计算MACD值
    newdata1['MACD'] = 2*(newdata1['DIFF_12_26'] - newdata1['DEA_12_26'])
    MD = []
    std_sum = 0
    for i in range(len(newdata1)):
        if i < 12:
            MD.append(0)
        else:
            for k in range(12):
                std_sum += (newdata1['Latestprice'].iloc[i-k] - newdata1['MA_12'].iloc[i-k])**2
            std_sum = np.sqrt(std_sum/12.0)
            MD.append(std_sum)
            std_sum = 0
# 计算上轨线
    newdata1['boll_up'] = newdata1['MA_12'] + 2*Series(MD)
# 计算下轨线
    newdata1['boll_down'] = newdata1['MA_12'] - 2*Series(MD)
# 计算%b指标
    newdata1['b_index'] = (newdata1['Latestprice'] - newdata1['boll_down'])/(newdata1['boll_up'] - newdata1['boll_down'])
# 计算通道宽度
    newdata1['channel_width'] = (newdata1['boll_up'] - newdata1['boll_down'])/newdata1['Latestprice']
    
    data=newdata1.ix[:,3:]
    where_are_nan = np.isnan(data)
    where_are_inf = np.isinf(data)
    data[where_are_nan] = 0
    data[where_are_inf] = 0
    
    data['MeanPrice'] = (data['Latestprice'] +data['MaxPrice']+data['MinPrice'])/3.0
    ser_1 = data['MeanPrice'][1:]
    ser_1.index = range(len(ser_1))
    ser1 = data['MeanPrice'][:-1]
    ser1.index = range(len(ser1))
    data['RaiseDown_1'] = (ser_1 -ser1)/ser1
# 同理，计算当前分钟的后2分钟、3分钟、4分钟、5分钟的涨跌幅(均相对于上一分钟)
    ser_2 = data['MeanPrice'][2:]
    ser_2.index = range(len(ser_2))
    ser2 = ser_1[:-1]
    ser2.index = range(len(ser2))
    data['RaiseDown_2'] = (ser_2 -ser2)/ser2

    ser_3 = data['MeanPrice'][3:]
    ser_3.index = range(len(ser_3))
    ser3 = ser_2[:-1]
    ser3.index = range(len(ser3))
    data['RaiseDown_3'] = (ser_3 -ser3)/ser3

    ser_4 = data['MeanPrice'][4:]
    ser_4.index = range(len(ser_4))
    ser4 = ser_3[:-1]
    ser4.index = range(len(ser4))
    data['RaiseDown_4'] = (ser_4 -ser4)/ser4

    ser_5 = data['MeanPrice'][5:]
    ser_5.index = range(len(ser_5))
    ser5 = ser_4[:-1]
    ser5.index = range(len(ser5))
    data['RaiseDown_5'] = (ser_5 -ser5)/ser5
    
    # 添加下一分钟的MA5、MA12、MA26、上轨线、下轨线指标
    ser_6 = data['MA_5'][1:]
    ser_6.index = range(len(ser_6))
    data['MA_5_PRED'] = ser_6

    ser_7 = data['MA_12'][1:]
    ser_7.index = range(len(ser_7))
    data['MA_12_PRED'] = ser_7

    ser_8 = data['MA_26'][1:]
    ser_8.index = range(len(ser_8))
    data['MA_26_PRED'] = ser_8

    ser_9 = data['boll_up'][1:]
    ser_9.index = range(len(ser_9))
    data['boll_up_PRED'] = ser_9

    ser_10 = data['boll_down'][1:]
    ser_10.index = range(len(ser_10))
    data['boll_down_PRED'] = ser_10
    
    data = data.fillna(method = 'ffill')
    data.index = range(len(data))
    data.to_csv(new_path + file_list[j].split('.')[0] + '_addfeature' + '.csv',sep=',') # 存储为加标签的数据
    
#     newdata1.to_csv(new_path + file_list[i].split('.')[0]+ '_features' + '.csv',sep=',')


## 制作数据集

In [4]:
path = '/home/chocolate/LSTM-source/Main_product_added_features/'
seq_new_25 = []
file_list = os.listdir(path)
for j in range(len(file_list)):
    data = pd.read_csv(path + file_list[j], header=0, sep=',',encoding="gbk")
    data_new = data.drop(data.columns[35:],axis = 1)
    data_new = data_new.ix[:, 1:]
#   数据归一化
    data_array_per = np.array(data_new)
    min_max_scaler = preprocessing.MinMaxScaler()
    data_array_per = min_max_scaler.fit_transform(data_array_per)
#  组时间序列
    for k in range(len(data_array_per)):
        if k < len(data_array_per)-5  and k >= 25:
            seq_new_25.append(data_array_per[k - 25:k])
seq_new_25 = np.array(seq_new_25)
print seq_new_25.shape
    

(334420, 25, 34)


## 为数据集打标签

In [3]:
# 打数据集标签,根据归一化后的涨跌情况打标签。
#但是不同合约归一化后的涨跌幅分布仍然不同，所以必须按合约分别打标签。
seq_label = []

# print u'IF合约:'
data_IF = pd.read_csv(path + 'IF_main_cleaned_addfeature.csv')
data_per_new = data_IF[25:]
data_per_new = data_per_new[:-5]   
 #   数据归一化
data_array_per = np.array(data_per_new)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#     print data_array_per
data_per = pd.DataFrame(data_array_per,columns = data_per_new.columns)
# print data_per['RaiseDown_1'].describe()
    
for i in data_per['RaiseDown_1']:
    if i > 0.74:
        seq_label.append([1,0,0,0,0,0,0])   #大涨
    elif i > 0.64:
        seq_label.append([0,1,0,0,0,0,0])   #小涨
    elif i> 0.52: 
        seq_label.append([0,0,1,0,0,0,0])   #平稳
    elif i > 0.4:
        seq_label.append([0,0,0,1,0,0,0])   #小跌
    elif i > 0.3:
        seq_label.append([0,0,0,0,1,0,0])   #大跌
    elif i > 0.2:
        seq_label.append([0,0,0,0,0,1,0])   #大跌
    else:
        seq_label.append([0,0,0,0,0,0,1])   #大跌

# print u'涨幅3：',seq_label.count([1,0,0,0,0,0,0]),seq_label.count([1,0,0,0,0,0,0]) /146399.0
# print u'涨幅2：',seq_label.count([0,1,0,0,0,0,0]),seq_label.count([0,1,0,0,0,0,0]) /146399.0
# print u'涨幅1：',seq_label.count([0,0,1,0,0,0,0]),seq_label.count([0,0,1,0,0,0,0]) /146399.0
# print u'平 稳：',seq_label.count([0,0,0,1,0,0,0]),seq_label.count([0,0,0,1,0,0,0]) /146399.0
# print u'跌幅1：',seq_label.count([0,0,0,0,1,0,0]),seq_label.count([0,0,0,0,1,0,0]) /146399.0
# print u'跌幅2：',seq_label.count([0,0,0,0,0,1,0]),seq_label.count([0,0,0,0,0,1,0]) /146399.0
# print u'跌幅3：',seq_label.count([0,0,0,0,0,0,1]),seq_label.count([0,0,0,0,0,0,1]) /146399.0


# print u'IC合约:'
data_IC = pd.read_csv(path + 'IC_main_cleaned_addfeature.csv')
data_per_new = data_IC[25:]
data_per_new = data_per_new[:-5]   
 #   数据归一化
data_array_per = np.array(data_per_new)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#     print data_array_per
data_per = pd.DataFrame(data_array_per,columns = data_per_new.columns)
# print data_per['RaiseDown_1'].describe()
    
for i in data_per['RaiseDown_1']:
    if i > 0.78:
        seq_label.append([1,0,0,0,0,0,0])   #大涨
    elif i > 0.67:
        seq_label.append([0,1,0,0,0,0,0])   #小涨
    elif i> 0.55: 
        seq_label.append([0,0,1,0,0,0,0])   #平稳
    elif i > 0.33:
        seq_label.append([0,0,0,1,0,0,0])   #小跌
    elif i > 0.23:
        seq_label.append([0,0,0,0,1,0,0])   #大跌
    elif i > 0.14:
        seq_label.append([0,0,0,0,0,1,0])   #大跌
    else:
        seq_label.append([0,0,0,0,0,0,1])   #大跌

# print u'涨幅3：',seq_label.count([1,0,0,0,0,0,0]),seq_label.count([1,0,0,0,0,0,0])/92931.0
# print u'涨幅2：',seq_label.count([0,1,0,0,0,0,0]),seq_label.count([0,1,0,0,0,0,0])/92931.0
# print u'涨幅1：',seq_label.count([0,0,1,0,0,0,0]),seq_label.count([0,0,1,0,0,0,0])/92931.0
# print u'平 稳：',seq_label.count([0,0,0,1,0,0,0]),seq_label.count([0,0,0,1,0,0,0])/92931.0
# print u'跌幅1：',seq_label.count([0,0,0,0,1,0,0]),seq_label.count([0,0,0,0,1,0,0])/92931.0
# print u'跌幅2：',seq_label.count([0,0,0,0,0,1,0]),seq_label.count([0,0,0,0,0,1,0])/92931.0
# print u'跌幅3：',seq_label.count([0,0,0,0,0,0,1]),seq_label.count([0,0,0,0,0,0,1])/92931.0

# print u'IH合约:'
data_IH = pd.read_csv(path + 'IH_main_cleaned_addfeature.csv')
data_per_new = data_IH[25:]
data_per_new = data_per_new[:-5]   
 #   数据归一化
data_array_per = np.array(data_per_new)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#     print data_array_per
data_per = pd.DataFrame(data_array_per,columns = data_per_new.columns)
# print data_per['RaiseDown_1'].describe()
    
for i in data_per['RaiseDown_1']:
    if i > 0.53:
        seq_label.append([1,0,0,0,0,0,0])   #大涨
    elif i > 0.463:
        seq_label.append([0,1,0,0,0,0,0])   #小涨
    elif i> 0.429: 
        seq_label.append([0,0,1,0,0,0,0])   #平稳
    elif i > 0.385:
        seq_label.append([0,0,0,1,0,0,0])   #小跌
    elif i > 0.355:
        seq_label.append([0,0,0,0,1,0,0])   #大跌
    elif i > 0.3:
        seq_label.append([0,0,0,0,0,1,0])   #大跌
    else:
        seq_label.append([0,0,0,0,0,0,1])   #大跌

# print u'涨幅3：',u'数量：',seq_label.count([1,0,0,0,0,0,0]),u'比例：',seq_label.count([1,0,0,0,0,0,0])/95090.0
# print u'涨幅2：',u'数量：',seq_label.count([0,1,0,0,0,0,0]),u'比例：',seq_label.count([0,1,0,0,0,0,0])/95090.0
# print u'涨幅1：',u'数量：',seq_label.count([0,0,1,0,0,0,0]),u'比例：',seq_label.count([0,0,1,0,0,0,0])/95090.0
# print u'平 稳：',u'数量：',seq_label.count([0,0,0,1,0,0,0]),u'比例：',seq_label.count([0,0,0,1,0,0,0])/95090.0
# print u'跌幅1：',u'数量：',seq_label.count([0,0,0,0,1,0,0]),u'比例：',seq_label.count([0,0,0,0,1,0,0])/95090.0
# print u'跌幅2：',u'数量：',seq_label.count([0,0,0,0,0,1,0]),u'比例：',seq_label.count([0,0,0,0,0,1,0])/95090.0
# print u'跌幅3：',u'数量：',seq_label.count([0,0,0,0,0,0,1]),u'比例：',seq_label.count([0,0,0,0,0,0,1])/95090.0

print u'涨幅3：',u'数量：',seq_label.count([1,0,0,0,0,0,0]),u'比例：',seq_label.count([1,0,0,0,0,0,0])/334420.0
print u'涨幅2：',u'数量：',seq_label.count([0,1,0,0,0,0,0]),u'比例：',seq_label.count([0,1,0,0,0,0,0])/334420.0
print u'涨幅1：',u'数量：',seq_label.count([0,0,1,0,0,0,0]),u'比例：',seq_label.count([0,0,1,0,0,0,0])/334420.0
print u'平 稳：',u'数量：',seq_label.count([0,0,0,1,0,0,0]),u'比例：',seq_label.count([0,0,0,1,0,0,0])/334420.0
print u'跌幅1：',u'数量：',seq_label.count([0,0,0,0,1,0,0]),u'比例：',seq_label.count([0,0,0,0,1,0,0])/334420.0
print u'跌幅2：',u'数量：',seq_label.count([0,0,0,0,0,1,0]),u'比例：',seq_label.count([0,0,0,0,0,1,0])/334420.0
print u'跌幅3：',u'数量：',seq_label.count([0,0,0,0,0,0,1]),u'比例：',seq_label.count([0,0,0,0,0,0,1])/334420.0
seq_label = np.array(seq_label)
seq_label.shape

涨幅3： 数量： 4031 比例： 0.012053704922
涨幅2： 数量： 9978 比例： 0.0298367322529
涨幅1： 数量： 41399 比例： 0.123793433407
平 稳： 数量： 224919 比例： 0.672564439926
跌幅1： 数量： 39501 比例： 0.11811793553
跌幅2： 数量： 10755 比例： 0.0321601578853
跌幅3： 数量： 3837 比例： 0.0114735960768


(334420, 7)

## 将数据打乱，提高预测准确率

In [4]:
from random import shuffle
zip_seq = zip(seq_new_25,seq_label)
shuffle(zip_seq)
seq_new_25,seq_label = zip(*zip_seq)
seq_new_25 = np.array(seq_new_25) 
seq_label = np.array(seq_label)
print seq_new_25.shape, seq_label.shape

(334420, 25, 34) (334420, 7)


## 划分训练集和测试集

In [5]:
train_data = seq_new_25[:234000]
train_label = seq_label[:234000]

test_data = seq_new_25[234000:]
test_label = seq_label[234000:]
train_data.shape, test_label.shape

((234000, 25, 34), (100420, 7))

## 保存训练集和测试集

In [ ]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
f1 = file(path_new + 'train_data.npy','wb')
np.save(f1,train_data)
f1.close()
f2 = file(path_new + 'test_data.npy','wb')
np.save(f2,test_data)
f2.close()
f3 = file(path_new + 'train_label.npy','wb')
np.save(f3,train_label)
f3.close()
f4 = file(path_new + 'test_label.npy','wb')
np.save(f4,test_label)
f4.close()

## 每次执行训练从此处开始

In [3]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
from sklearn import preprocessing
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 读取训练集和测试集

In [ ]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
# 读取训练集和测试集
f1 = file(path_new + 'train_data.npy','rb')
train_data = np.load(f1)
f2 = file(path_new + 'test_data.npy','rb')
test_data = np.load(f2)
f3 = file(path_new + 'train_label.npy','rb')
train_label = np.load(f3)
f4 = file(path_new + 'test_label.npy','rb')
test_label = np.load(f4)

## 构建LSTM神经网络，执行训练并保存模型

In [8]:
import random

learning_rate = 0.001 # 学习速率
training_iters = 3000000  # 训练总步长
batch_size = 1500 # 每小段步长
display_step = 100  # 每隔100小段显示输出

# 参数设置
n_input = 34 # 特征数量
n_steps = 25 # 时间序列长度
n_hidden = 500 # 隐藏层神经元个数
n_classes = 7 # 分类数量

tf.reset_default_graph()  # 重置流图
# 设置输入输出格式大小
xtr = tf.placeholder("float", [None, n_steps, n_input])
ytr = tf.placeholder("float", [None,n_classes])

# 定义权重和偏移量
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# def pred():
    
def RNN(x, weights, biases):

    # 更改数据集的格式，使其满足RNN网络的输入需求
    # 现在输入格式: (batch_size, n_steps, n_input)
    # 需要的格式: 'n_steps' 个tensor，每个的格式为 (batch_size, n_input)

    # 变换batch_size 和 n_steps的位置
    x = tf.transpose(x, [1, 0, 2])
    # 重新调整格式 (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # 分割成一系列的 'n_steps' 个tensors，每个的格式为 (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # 定义LSTM神经网络结构
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # 获取输出和状态
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)
    # 返回输出
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
# 获取预测值
pred = RNN(xtr, weights, biases)
# train_label = tf.split(0, batch_size, train_label)
# 取下一小段的输入
def next_batch_tr(batch_size):
    step = random.randint(1, len(train_data)/batch_size)
    next_batch_data = train_data[(step-1)*batch_size:step*batch_size]
    next_batch_label = train_label[(step-1)*batch_size:step*batch_size]   
    return next_batch_data, next_batch_label



In [9]:
# 定义代价函数和最优化方法
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, ytr))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 评估模型准确率
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(ytr, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 变量初始化
init = tf.initialize_all_variables()
part_saver = tf.train.Saver()
# 创建会话
sess = tf.InteractiveSession()
sess.run(init)
step = 1
    # 一直迭代，直到最大步长
while step * batch_size < training_iters:
    batch_x, batch_y = next_batch_tr(batch_size)
    batch_x = batch_x.reshape((-1, n_steps, n_input))
        # 利用最优化方法调整权值
    sess.run(optimizer, feed_dict={xtr: batch_x, ytr: batch_y})
    if step % display_step == 0:
            # 计算准确率和损失函数
        acc = sess.run(accuracy, feed_dict={xtr: batch_x, ytr: batch_y})
        loss = sess.run(cost, feed_dict={xtr: batch_x, ytr: batch_y})
        # 每隔一定步长显示训练准确率和损失率
        print("Iter " + str(step * batch_size) + ", Minibatch Loss= " +
                "{:.6f}".format(loss) + ", Training Accuracy= " +
                "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")

saver_path = part_saver.save(sess, "/home/chocolate/Model_LSTM-Future_20161123/models/predict_1min.ckpt")
print "Model saved." 


Iter 150000, Minibatch Loss= 0.993739, Training Accuracy= 0.65867
Iter 300000, Minibatch Loss= 0.960534, Training Accuracy= 0.66200
Iter 450000, Minibatch Loss= 0.937401, Training Accuracy= 0.67667
Iter 600000, Minibatch Loss= 0.990155, Training Accuracy= 0.63933
Iter 750000, Minibatch Loss= 0.915748, Training Accuracy= 0.68200
Iter 900000, Minibatch Loss= 0.932645, Training Accuracy= 0.65533
Iter 1050000, Minibatch Loss= 0.985310, Training Accuracy= 0.67400
Iter 1200000, Minibatch Loss= 0.916208, Training Accuracy= 0.67067
Iter 1350000, Minibatch Loss= 0.951309, Training Accuracy= 0.66333
Iter 1500000, Minibatch Loss= 0.899215, Training Accuracy= 0.68000
Iter 1650000, Minibatch Loss= 0.914494, Training Accuracy= 0.66467
Iter 1800000, Minibatch Loss= 0.897618, Training Accuracy= 0.66533
Iter 1950000, Minibatch Loss= 0.931017, Training Accuracy= 0.64600
Iter 2100000, Minibatch Loss= 0.863796, Training Accuracy= 0.68133
Iter 2250000, Minibatch Loss= 0.880785, Training Accuracy= 0.68933
I

## 加载模型，并对测试集进行准确率测试

In [1]:
saver = tf.train.Saver()
saver.restore(sess, "/home/chocolate/Model_LSTM-Future_20161123/models/predict_1min.ckpt")
print "Model restored."
test_data1 = test_data.reshape((-1, n_steps, n_input))
# test_label1 = test_label[:test_len]
print("Testing Accuracy:",
        sess.run(accuracy, feed_dict={xtr: test_data1, ytr: test_label}))


NameError: name 'tf' is not defined